# MLP Coding Example

Acesse o servidor remoto por ssh. Crie uma virtualenv com:
```
mkvirtualenv <nome-da-sua-env>
```
Ative a sua virtualenv com:
```
workon <nome-da-sua-env>
```
Instale o jupyter:
```
pip install jupyter
```
Na pasta contendo o setup.py, instale o pacote do projeto :
```
pip install -e .
```
Comando para servir o jupyter:
```
nohup jupyter notebook --no-browser &
```
Talvez você precise de um token. Se precisar consulte com:
```
jupyter notebook list
```




Na sua máquina local, redirecione a porta adequada:
```
ssh -NfL localhost:<porta-local>:localhost:<porta-remoto> <seu-usuario>@<ip-do-servidor>
```
Geralmente:
```
ssh -NfL localhost:8888:localhost:8888 <seu-usuario>@<ip-do-servidor>
```
Abra localhost:8888 no seu browser. Se você quiser fechar o jupyter, no localhost:8888 clique em Quit, depois libere a porta com:
```
lsof -ti:8888 | xargs kill -9
```

## Imports

In [1]:
import numpy as np
from tqdm.notebook import tqdm
from perceptronac.context_training import context_training
from perceptronac.context_coding import context_coding
from perceptronac.perfect_AC import perfect_AC
import torch

## Gerando dados randômicos correlacionados (substituir pelos seus dados)

In [2]:
# parameters  
L = 100000 # how many samples 
N = 7 # order of the AR
# Np = N # number of parameters to estimate 

C0 = np.random.rand(1,1) 
C = np.random.rand(N,1)

X = 2 * (np.random.rand(2*L,N) > 0.5) - 1 # correlated (context) signals

def sigmoid(x): 
    return 1 / (1 + np.e**(-x))

p = sigmoid(C0 + X @ C);
yy = (np.random.rand(2*L, 1) > (1 - p)); # signal 
yt = (yy[0:L] > 0).astype(int) # train on the first part 
yc = (yy[L:L+L] > 0).astype(int); # encode the second part
Xt = (X[0:L,0:N] > 0).astype(int); # truncated X for training 
Xc = (X[L:L+L,0:N] > 0).astype(int); # truncated X for coding

## Entropia dos dados

In [3]:
# treino
perfect_AC(yt,context_coding(Xt,context_training(Xt,yt)))

0.7278209104289146

In [5]:
# teste
perfect_AC(yc,context_coding(Xc,context_training(Xc,yc)))

0.7248627782756322

## Treinando Modelo No Pytorch com Batch Gradient Descent (Quando todos os dados couberem na memória da placa de vídeo de uma só vez)

In [6]:
import torch

In [7]:
class Perceptron(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.linear = torch.nn.Linear(N, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [8]:
class Log2BCELoss(torch.nn.Module):
    def __init__(self,*args,**kwargs):
        super().__init__()
        self.bce_loss = torch.nn.BCELoss(*args,**kwargs)

    def forward(self, pred, target):
        return self.bce_loss(pred, target)/torch.log(torch.tensor(2,dtype=target.dtype,device=target.device))

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx,:]

In [10]:
net = Perceptron(N)

In [11]:
net = net.cuda()

In [12]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [14]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [15]:
train_loss = []
for epoch in range(20000):
    optimizer.zero_grad()
    outputs = net(torch.tensor(trainset.X).float().cuda())
    loss = criterion(outputs,torch.tensor(trainset.y).view(-1,1).float().cuda())
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item()/len(trainset))

In [26]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

comprimento médio de código final no dataset de treino: 0.728532890625
(compare com a entropia do dataset de treino).


### Pesos aprendidos estão relacionados com os parâmetros usados para gerar os dados por um fator de escala, com o bias ajustado para corrigir a diferença

In [27]:
for param in net.parameters():
    print(param.data)

tensor([[0.2242, 0.9829, 0.2125, 0.2316, 1.4047, 1.8878, 1.7824]],
       device='cuda:0')
tensor([-2.9886], device='cuda:0')


In [30]:
C.T, C0

(array([[0.10138062, 0.49044468, 0.09653812, 0.11257858, 0.71199488,
         0.93575754, 0.88620181]]),
 array([[0.37240823]]))

## Treinando Modelo No Pytorch com Stochastic Gradient Descent (um pedaço dos dados na memória da placa de vídeo de cada vez)

In [31]:
net = Perceptron(N)

In [32]:
net = net.cuda()

In [33]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [34]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [35]:

batch_size = 100000

train_loss, valid_loss = [], []

for epoch in range(200):  # loop over the dataset multiple times

    for phase in ['train','valid']:

        if phase == 'train':
            net.train(True)
            dataloader = torch.utils.data.DataLoader(
                trainset,batch_size=batch_size,shuffle=True,num_workers=2)
        else:
            net.train(False)
            dataloader=torch.utils.data.DataLoader(
                validset,batch_size=batch_size,shuffle=False,num_workers=2)
            
        running_loss = 0.0
        for data in dataloader: #tqdm(dataloader):
            
            X_b,y_b= data
            X_b = X_b.float().cuda()
            y_b = y_b.float().cuda()
            
            if phase == 'train':
                optimizer.zero_grad()
                outputs = net(X_b.float())
                loss = criterion(outputs,y_b.view(-1,1).float())
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    outputs = net(X_b)
                    loss = criterion(outputs, y_b)

            running_loss += loss.item()

        final_loss = running_loss / len(dataloader.dataset)
        if phase=='train':
            train_loss.append(final_loss)
        else:
            valid_loss.append(final_loss)
            
        print("epoch :" , epoch, ", phase :", phase, ", loss :", final_loss)

print('Finished Training')

epoch : 0 , phase : valid , loss : 0.99760828125
epoch : 1 , phase : valid , loss : 0.97324859375
epoch : 2 , phase : valid , loss : 0.9519415625
epoch : 3 , phase : valid , loss : 0.933254296875
epoch : 4 , phase : valid , loss : 0.91680921875
epoch : 5 , phase : valid , loss : 0.90228078125
epoch : 6 , phase : valid , loss : 0.889391875
epoch : 7 , phase : valid , loss : 0.877906953125
epoch : 8 , phase : valid , loss : 0.867627890625
epoch : 9 , phase : valid , loss : 0.858387578125
epoch : 10 , phase : valid , loss : 0.850045234375
epoch : 11 , phase : valid , loss : 0.84248234375
epoch : 12 , phase : valid , loss : 0.83559890625
epoch : 13 , phase : valid , loss : 0.829310234375
epoch : 14 , phase : valid , loss : 0.823544453125
epoch : 15 , phase : valid , loss : 0.81824046875
epoch : 16 , phase : valid , loss : 0.813346328125
epoch : 17 , phase : valid , loss : 0.808816953125
epoch : 18 , phase : valid , loss : 0.80461390625
epoch : 19 , phase : valid , loss : 0.800703828125
epo

epoch : 163 , phase : valid , loss : 0.726224140625
epoch : 164 , phase : valid , loss : 0.726208359375
epoch : 165 , phase : valid , loss : 0.72619296875
epoch : 166 , phase : valid , loss : 0.726178046875
epoch : 167 , phase : valid , loss : 0.72616359375
epoch : 168 , phase : valid , loss : 0.72614953125
epoch : 169 , phase : valid , loss : 0.7261359375
epoch : 170 , phase : valid , loss : 0.726122734375
epoch : 171 , phase : valid , loss : 0.72610984375
epoch : 172 , phase : valid , loss : 0.726097421875
epoch : 173 , phase : valid , loss : 0.726085390625
epoch : 174 , phase : valid , loss : 0.726073671875
epoch : 175 , phase : valid , loss : 0.726062265625
epoch : 176 , phase : valid , loss : 0.726051171875
epoch : 177 , phase : valid , loss : 0.726040546875
epoch : 178 , phase : valid , loss : 0.72603015625
epoch : 179 , phase : valid , loss : 0.726020078125
epoch : 180 , phase : valid , loss : 0.726010234375
epoch : 181 , phase : valid , loss : 0.726000625
epoch : 182 , phase : 

In [36]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

print(f"""comprimento médio de código final no dataset de validação: {valid_loss[-1]}
(compare com a entropia do dataset de validação).""")

comprimento médio de código final no dataset de treino: 0.72873578125
(compare com a entropia do dataset de treino).
comprimento médio de código final no dataset de validação: 0.7258703125
(compare com a entropia do dataset de validação).


### Pesos aprendidos estão relacionados com os parâmetros usados para gerar os dados por um fator de escala, com o bias ajustado para corrigir a diferença

In [37]:
for param in net.parameters():
    print(param.data)

tensor([[0.1999, 0.9447, 0.1883, 0.2068, 1.3592, 1.8354, 1.7316]],
       device='cuda:0')
tensor([-2.8569], device='cuda:0')


In [38]:
C.T, C0

(array([[0.10138062, 0.49044468, 0.09653812, 0.11257858, 0.71199488,
         0.93575754, 0.88620181]]),
 array([[0.37240823]]))